# Data Science Project SoSe 2024
## Team 07
- Maximilian Hoffmann
- Kilian Kempf
- Daniel Schneider
- Tom Schuck

## Project Submission

### Data Initialization

In [1]:
import os

import pandas as pd

from data_management import DataManager

DATA_DIR = os.path.join(os.getcwd(), 'data/Instacart')

op_prior = pd.read_csv(os.path.join(DATA_DIR, 'order_products__prior.csv.zip'))
op_train = pd.read_csv(os.path.join(DATA_DIR, 'order_products__train.csv.zip'))

tip_train = pd.read_csv(os.path.join(DATA_DIR, 'tip_trainingsdaten1_.csv'))[['order_id', 'tip']]
tip_test = pd.read_csv(os.path.join(DATA_DIR, 'tip_testdaten1_template.csv'))

orders = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv.zip'))
aisles = pd.read_csv(os.path.join(DATA_DIR, 'aisles.csv.zip'))
departments = pd.read_csv(os.path.join(DATA_DIR, 'departments.csv.zip'))
products = pd.read_csv(os.path.join(DATA_DIR, 'products.csv.zip'))

data_manager = DataManager(op_prior, op_train, tip_train, tip_test, orders, products, aisles, departments)
order_amount = len(data_manager.get_orders_tip())

### Feature Engineering

In [2]:
from feature_engineering.static_features import TipHistory, ReorderedRatio, OrderSize, PrevTippedProductsRatio, \
    CustomerLifetime, PrevOrderTipped, OrderFrequency, SimOrdersTipRatio, AvgSizePrevOrders, MeanOrderedRate, \
    LastTipSequence, RelDaysSinceTip, DaysSinceTip, OrderNumberSquared, HodHighTipProbability, DowHighTipProbability, ContainsAlcohol

from feature_engineering.dynamic_features import ProductTipRate, DepartmentTipRate, AisleTipRate, DynamicFeatureTest1, \
    DynamicFeatureTest2, AssocRulesAisles, AssocRulesDepartments

# Kilian
tip_history = TipHistory()
reordered_rate = ReorderedRatio()
order_size = OrderSize()
prev_tipped_products_ratio = PrevTippedProductsRatio()
customer_lifetime = CustomerLifetime()
prev_order_tipped = PrevOrderTipped()

# Daniel
order_frequency = OrderFrequency()
mean_ordered_rate = MeanOrderedRate()
rel_days_since_tip = RelDaysSinceTip()
days_since_tip = DaysSinceTip()

# Max
sim_orders_tip_ratio = SimOrdersTipRatio()
product_tip_rate = ProductTipRate()
department_tip_rate = DepartmentTipRate()
aisle_tip_rate = AisleTipRate()
last_tip_sequence = LastTipSequence()

# Tom
avg_size_prev_orders = AvgSizePrevOrders()

# Features analysis
order_number_squared = OrderNumberSquared()
hod_high_tip_probability = HodHighTipProbability()
dow_high_tip_probability = DowHighTipProbability()
contains_alcohol = ContainsAlcohol()

assoc_rules_departments = AssocRulesDepartments()
assoc_rules_aisles = AssocRulesAisles()

# Test
dynamic_feature_test_1 = DynamicFeatureTest1()
dynamic_feature_test_2 = DynamicFeatureTest2()

In [3]:
# Static Features
data_manager.register_feature(tip_history)
data_manager.register_feature(reordered_rate)
data_manager.register_feature(order_size)
data_manager.register_feature(customer_lifetime)
data_manager.register_feature(prev_order_tipped)
data_manager.register_feature(prev_tipped_products_ratio)

data_manager.register_feature(order_frequency)
data_manager.register_feature(sim_orders_tip_ratio)
data_manager.register_feature(avg_size_prev_orders)
data_manager.register_feature(mean_ordered_rate)
data_manager.register_feature(last_tip_sequence)
data_manager.register_feature(rel_days_since_tip)
data_manager.register_feature(days_since_tip)

data_manager.register_feature(order_number_squared)
data_manager.register_feature(hod_high_tip_probability)
data_manager.register_feature(dow_high_tip_probability)
data_manager.register_feature(contains_alcohol)

# Dynamic Features
data_manager.register_feature(dynamic_feature_test_1)
data_manager.register_feature(dynamic_feature_test_2)
data_manager.register_feature(product_tip_rate)
data_manager.register_feature(department_tip_rate)
data_manager.register_feature(aisle_tip_rate)

data_manager.register_feature(assoc_rules_departments)
data_manager.register_feature(assoc_rules_aisles)

In [4]:
data_manager.compute_features()

In [5]:
data_manager.get_orders_tip().head(100)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,tip_history,order_size,dynamic_feature_test_2,dynamic_feature_test_1
0,2539329,1,prior,1,2,8,NaN,0.0,NaN,5,0.090909,0.090909
1,2398795,1,prior,2,3,7,15.0,0.0,0.0,6,0.181818,0.181818
2,473747,1,prior,3,3,12,21.0,0.0,0.0,5,0.272727,0.272727
3,2254736,1,prior,4,4,7,29.0,0.0,0.0,5,0.363636,0.363636
4,431534,1,prior,5,4,15,28.0,0.0,0.0,8,0.454545,0.454545
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1916106,12,prior,3,5,8,14.0,0.0,0.0,12,0.600000,0.600000
96,1057378,12,prior,4,3,9,26.0,0.0,0.0,20,0.800000,0.800000
97,221248,12,prior,5,1,9,30.0,0.0,0.0,22,1.000000,1.000000
98,2618231,13,prior,1,6,12,NaN,0.0,NaN,5,0.076923,0.076923


In [6]:
data_manager.export_features('data/prepared_data/computed_features.csv.zip', only_static=False)

In [3]:
data_manager.import_features('data/prepared_data/computed_features.csv.zip', only_static=False)

In [8]:
# data_manager.get_orders_tip().head(100)

In [9]:
# test = data_manager.get_orders_tip()

In [10]:
# print(f"Number of orders: {len(data_manager.get_orders_tip())}")
# print(f"Number of orders did not change: {order_amount == len(data_manager.get_orders_tip())}\n")
# print(f"Number of NaN or null values in each column:\n{data_manager.get_orders_tip().isnull().sum()}")

In [11]:
# order_ids = data_manager.get_orders_tip(complete=True).groupby('user_id')['order_id'].head(5)
# order_amount = len(order_ids)

In [12]:
# data_manager.set_subset(order_ids)

In [13]:
# data_manager.get_orders_tip().head(25)

In [14]:
# print(f"Number of orders: {len(data_manager.get_orders_tip())}")
# print(f"Number of orders did not change: {order_amount == len(data_manager.get_orders_tip())}\n")
# print(f"Number of NaN or null values in each column:\n{data_manager.get_orders_tip().isnull().sum()}")

In [15]:
# order_ids = data_manager.get_orders_tip(complete=True).groupby('user_id')['order_id'].head(4)
# order_amount = len(order_ids)
# 
# data_manager.set_subset(order_ids)
# data_manager.get_orders_tip().head(25)

In [16]:
# print(f"Number of orders: {len(data_manager.get_orders_tip())}")
# print(f"Number of orders did not change: {order_amount == len(data_manager.get_orders_tip())}\n")
# print(f"Number of NaN or null values in each column:\n{data_manager.get_orders_tip().isnull().sum()}")

### Analysis
- Only orders from the training set (orders with tip information) are considered

In [4]:
# from analysis import DayOfWeek, HourOfDay, DaysSincePriorOrder, OrderNumber, Department, Aisle, Product, GeneralAnalysis

In [5]:
# day_of_week = DayOfWeek(data_manager)
# hour_of_day = HourOfDay(data_manager)
# days_since_prior_order = DaysSincePriorOrder(data_manager)
# order_number = OrderNumber(data_manager)
# department = Department(data_manager)
# aisle = Aisle(data_manager)
# product = Product(data_manager)
# number_order_user = NumberOrderUser(data_manager)
# general_analysis = GeneralAnalysis(data_manager)

In [19]:
# day_of_week.execute_analysis()

In [20]:
# hour_of_day.execute_analysis()

In [21]:
# days_since_prior_order.execute_analysis()

In [22]:
# order_number.execute_analysis()

In [23]:
# department.execute_analysis()

In [24]:
# aisle.execute_analysis()

In [25]:
# product.execute_analysis()

In [26]:
# number_order_user.execute_analysis()

In [6]:
# general_analysis.execute_analysis()

General facts about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819106 entries, 0 to 33819105
Data columns (total 16 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
 7   tip                     object 
 8   product_id              int64  
 9   add_to_cart_order       int64  
 10  reordered               int64  
 11  product_name            object 
 12  aisle_id                int64  
 13  department_id           int64  
 14  aisle                   object 
 15  department              object 
dtypes: float64(1), int64(10), object(5)
memory usage: 4.0+ GB
None
(33819106, 16)


Size of the Dataset:
Number of prior Data 32434489
Number of train Data 1384617


General Information

KeyboardInterrupt: 

### Data Preparation

In [27]:
# from feature_engineering.static_features import DowHighTipProbability, HodHighTipProbability, OrderNumberSquared, \
#     ContainsAlcohol
#
# data_manager.register_feature(DowHighTipProbability())
# data_manager.register_feature(HodHighTipProbability())
# data_manager.register_feature(OrderNumberSquared())
# data_manager.register_feature(ContainsAlcohol())

In [28]:
# data_manager.compute_features()

### Model Training & Evaluation

In [29]:
from data_management.cross_validation import LastOrderUserTSCVSplitter

In [30]:
orders_tip = data_manager.get_orders_tip()
orders_tip.shape

(3346083, 12)

In [31]:
order_ids = orders_tip[orders_tip['order_number'] > 1]['order_id']
# order_ids = orders_tip['order_id']
data_manager.set_subset(order_ids)

In [32]:
orders_tip_train = data_manager.get_orders_tip_train()
orders_tip_train.shape

(3008665, 12)

In [33]:
# cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [34]:
last_order_user_tscv_splitter = LastOrderUserTSCVSplitter(data_manager, n_splits=5)

In [35]:
# splits = last_order_user_tscv_splitter.split(orders_tip_train)

In [36]:
# folds = {}
# prev_test = 0
# for i, (train_index, test_index) in enumerate(splits):
#     print(f'Fold {i + 1}')
#     print(f'Train: {len(train_index)}')
#     print(f'Test: {len(test_index)}')
#     print(f'Orders: {len(orders_tip_train)}')
#     print(f'Sum equal: {(len(train_index) + len(test_index) + prev_test) == len(orders_tip_train)}')
#     print(f'Ratio: {len(test_index) / (len(train_index) + len(test_index))}')
#
#     train_orders = orders_tip_train.loc[train_index]
#     test_orders = orders_tip_train.loc[test_index]
#     current_fold = {
#         'train': train_orders,
#         'test': test_orders
#     }
#     folds[f'fold_{i}'] = current_fold
#     prev_test += len(test_index)

In [37]:
last_order_user_tscv_splitter.export_splits('data/prepared_data/')

Export 1: Train size: 2802456, Test size: 206209
Export 2: Train size: 2596247, Test size: 206209
Export 3: Train size: 2414024, Test size: 182223
Export 4: Train size: 2251391, Test size: 162633
Export 5: Train size: 2104923, Test size: 146468


In [38]:
# user_tscv_splitter = UserTSCVSplitter(data_manager, n_splits=5, validation_set_ratio=0.2, seed=42)

In [39]:
# splits = user_tscv_splitter.split(orders_tip_train)

In [40]:
# folds = {}
# prev_test = 0
# for i, (train_index, test_index) in enumerate(splits):
#     print(f'Fold {i + 1}')
#     print(f'Train: {len(train_index)}')
#     print(f'Test: {len(test_index)}')
#     print(f'Orders: {len(orders_tip_train)}')
#     print(f'Sum equal: {(len(train_index) + len(test_index) + prev_test) == len(orders_tip_train)}')
#     print(f'Ratio: {len(test_index) / (len(train_index) + len(test_index))}')
# 
#     train_orders = orders_tip_train.loc[train_index]
#     test_orders = orders_tip_train.loc[test_index]
#     current_fold = {
#         'train': train_orders,
#         'test': test_orders
#     }
#     folds[f'fold_{i}'] = current_fold
#     prev_test += len(test_index)

In [41]:
# user_tscv_splitter.export_splits('data/prepared_data/')

In [42]:
# features = ['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'order_size', 'tip_history',
#             'dynamic_feature_test_1', 'dynamic_feature_test_2']
#
# prepared_splits_dict = user_tscv_splitter.import_splits('data/prepared_data/', features)

In [43]:
features = ['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'order_size', 'tip_history',
            'dynamic_feature_test_1', 'dynamic_feature_test_2']

prepared_splits_dict = last_order_user_tscv_splitter.import_splits('data/prepared_data/', features)

In [44]:
from data_management import DatasetSelector
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from tempfile import mkdtemp
from joblib import Memory

pipeline_steps = [('selector', DatasetSelector(prepared_splits_dict)),
                  ('estimator', DecisionTreeClassifier())]

cachedir = mkdtemp()
memory = Memory(location=cachedir, verbose=0)

pipeline = Pipeline(pipeline_steps, verbose=False, memory=memory)

In [45]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'estimator__criterion': ['entropy'],
    'estimator__max_depth': [9, 10],
    'estimator__min_samples_leaf': [256]
}

grid_search_clf = GridSearchCV(pipeline, param_grid, cv=last_order_user_tscv_splitter, scoring='accuracy', verbose=1,
                               n_jobs=-1)

In [46]:
X = orders_tip_train[features]
y = orders_tip_train['tip'].astype(bool)

In [47]:
X.head(25)

,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_size,tip_history,dynamic_feature_test_1,dynamic_feature_test_2
0,2,3,7,15.0,6,0.000000,0.100000,0.100000
1,3,3,12,21.0,5,0.000000,0.200000,0.200000
2,4,4,7,29.0,5,0.000000,0.300000,0.300000
3,5,4,15,28.0,8,0.000000,0.400000,0.400000
4,6,2,7,19.0,4,0.000000,0.500000,0.500000
5,7,1,9,20.0,5,0.000000,0.600000,0.600000
6,8,1,14,14.0,6,0.000000,0.700000,0.700000
7,9,1,16,0.0,6,0.000000,0.800000,0.800000
8,10,4,8,30.0,9,0.000000,0.900000,0.900000
9,2,5,10,10.0,6,0.000000,0.071429,0.071429


In [ ]:
grid_search_clf.fit(X, y)

/Users/I538977/Library/Mobile Documents/com~apple~CloudDocs/2. Semester Master/Data Science/Abgabe/data-science-assignment/.venv/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=59266) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()


Fitting 5 folds for each of 2 candidates, totalling 10 fits


/Users/I538977/Library/Mobile Documents/com~apple~CloudDocs/2. Semester Master/Data Science/Abgabe/data-science-assignment/.venv/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=59266) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/I538977/Library/Mobile Documents/com~apple~CloudDocs/2. Semester Master/Data Science/Abgabe/data-science-assignment/.venv/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=59266) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/I538977/Library/Mobile Documents/com~apple~CloudDocs/2. Semester Master/Data Science/Abgabe/data-science-assignment/.venv/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=59266) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid =

In [ ]:
cv_results = grid_search_clf.cv_results_
result_list = [grid_search_clf.cv_results_[f'split{i}_test_score'][grid_search_clf.best_index_] for i in range(5)]

for i, result in enumerate(result_list):
    print(f'Accuracy (Fold {i + 1}): {result}')

print(f'\nMean Accuracy: {cv_results["mean_test_score"][grid_search_clf.best_index_]}')

In [ ]:
grid_search_clf.best_score_

In [ ]:
grid_search_clf.best_params_

In [ ]:
best_estimator = grid_search_clf.best_estimator_

### Prediction

In [ ]:
# orders_tip_test = data_manager.get_orders_tip_test()
# orders_tip_test['tip'] = best_estimator.predict(orders_tip_test[features])

In [ ]:
# orders_tip_test_csv = orders_tip_test[tip_test.columns].copy()
# orders_tip_test_csv.rename(columns={tip_test.columns[0]: ''}, inplace=True)
# orders_tip_test_csv

In [ ]:
# orders_tip_test_csv.to_csv(os.path.join(DATA_DIR, 'tip_testdaten1.csv'), index=False)